In [2]:
from tensorflow import keras
from keras import models
from keras import layers
from keras import optimizers, losses, metrics
from keras import preprocessing

import numpy as np
import pandas as pd
import pickle
import re

from konlpy.tag import Okt

In [3]:
# 태그 단어
PAD = "<PADDING>"   # 패딩
STA = "<START>"     # 시작
END = "<END>"       # 끝
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)

# 태그 인덱스
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

# 데이터 타입
ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수
max_sequences = 30

# 정규 표현식 필터
RE_FILTER = re.compile("[.,!?\"':;~()]")

In [4]:
chatbot_data = pd.read_csv('./train.csv')
chatbot_data.head(3)

,Q,A
0,보관만 하는데 9만 원인가요 락커는 얼마죠,대형은 28만 원 중형은 19만 원 소형은 15만 원입니다
1,#@소속#이요 이름이 정겹네요,그 식당에서 드셔 보시면 홍천하면 보리밥부터 생각나실 거예요
2,#@소속# 공원 근처에 맛있는 분식집 추천해 주세요,국물 떡볶이가 있는 #@소속#을 추천해 드립니다


In [5]:
question, answer = list(chatbot_data['Q']), list(chatbot_data['A'])

In [6]:
# 데이터 개수
len(question), len(answer)

(33994, 33994)

In [7]:
# 일부만 학습에 사용
question = question[:1000]
answer = answer[:1000]

for i in range(5):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()

Q : 보관만 하는데 9만 원인가요 락커는 얼마죠 
A : 대형은 28만 원 중형은 19만 원 소형은 15만 원입니다 

Q : #@소속#이요 이름이 정겹네요 
A : 그 식당에서 드셔 보시면 홍천하면 보리밥부터 생각나실 거예요 

Q : #@소속# 공원 근처에 맛있는 분식집 추천해 주세요 
A : 국물 떡볶이가 있는 #@소속#을 추천해 드립니다 

Q : 실내에도 다양하게 준비되어 있네요 
A : 그러니 직접 체험해 보시길 바랍니다 

Q : 이번에 김포 여행을 가려고 하는데요 
A : 무엇을 도와드릴까요 



In [8]:
question

['보관만 하는데 9만 원인가요 락커는 얼마죠 ',
 '#@소속#이요 이름이 정겹네요 ',
 '#@소속# 공원 근처에 맛있는 분식집 추천해 주세요 ',
 '실내에도 다양하게 준비되어 있네요 ',
 '이번에 김포 여행을 가려고 하는데요 ',
 '아무것도 없는 줄 알았어요 ',
 '#@소속# 맞습니다 ',
 '배를 타는 시간이 있나요 ',
 '절매 나루터에서 진탄 나루터까지 하는 코스 맞지요 ',
 '#@소속#에서 받으면 되는 거죠 ',
 '그건 따로 못 보긴 했는데 수요일만 쉬는 건가요 ',
 '영월 10경 중에 사람이 지은 건축물 같은 거 말구요 강가 따라서 풍경 볼 수 있는 곳 추천 좀 해주세요 ',
 '#@소속# 예약할 수 있나 확인하려구요 ',
 '녹두한방백숙은 가격이 얼마예요 ',
 '건물 뒤에 있어서 주차장이 안 보였군요 앞에 차들은 뭐예요 ',
 '체험할 수 있는 프로그램이 있습니까 ',
 '친절한 안내 감사드립니다 ',
 '강습은 받지 않고 장비 렌탈만 이용할 수는 없나요 ',
 '땀이 많은 편이라 복장 때문에 문의드려요 등산복 긴팔에 구스 경량패딩을 입고 갈려고 하거든요 ',
 '제가 관심은 있는데 전혀 할 줄 몰라서요 ',
 '제 이메일이요 #@계정#입니다 ',
 '40분이나 기다려야 하네요 ',
 '수상스키를 체험하고 싶은데요 금액이 어떻게 되나요 ',
 '이천 여행 중인데 맛집을 찾고 있어요 영업 시간이 어떻게 되나요 ',
 '안녕하세요 저희가 오늘 여행 마지막 날인데요 불꽃축제 보고 엑스포역에서 바로 기차를 타야 하거든요 ',
 '진해 군항제를 다녀와야겠네요 ',
 '엑스포공원갔다가 돌머리 해수욕장 들러서 해수찜 하러 갈거에요 ',
 '방금 예약했던 게 취소가되서 확인 좀 부탁 드리려 구요 ',
 '주말에 단독으로 요트투어가 가능하나요 ',
 '대표 메뉴가 무엇인가요 ',
 '초등학생 체험학습도 있나요 ',
 '공룡발자국을 보려면 백이산 어느 코스로 가야 하나요 ',
 '이용요금도 알 수 있을까요 ',
 '문어도 숯불에 구워 주시나요 ',

In [9]:
# 형태소 분석 함수
def pos_tag(sentences):

    tagger = Okt()

    # 문장 품사 변수 초기화
    sentences_pos = []

    # 모든 문장 반복
    for sentence in sentences:
        sentence = re.sub(RE_FILTER, "", sentence)

        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
        
    return sentences_pos

In [10]:
question = pos_tag(question)
answer = pos_tag(answer)

In [11]:
for i in range(5):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()

Q : 보관 만 하는데 9만 원 인가요 락커 는 얼마 죠
A : 대형 은 28만 원 중형 은 19만 원 소형 은 15만 원 입니다

Q : #@ 소속 # 이요 이름 이 정겹네요
A : 그 식당 에서 드셔 보시 면 홍천 하면 보리밥 부터 생각나실 거 예요

Q : #@ 소속 # 공원 근처 에 맛있는 분식집 추천 해 주세요
A : 국물 떡볶이 가 있는 #@ 소속 # 을 추천 해 드립니다

Q : 실내 에도 다양하게 준비 되어 있네요
A : 그러니 직접 체험 해 보시길 바랍니다

Q : 이번 에 김포 여행 을 가려고 하는데요
A : 무엇 을 도 와 드릴 까요



In [12]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(question)
sentences.extend(answer)

words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

In [13]:
print(words)

['보관', '만', '하는데', '9만', '원', '인가요', '락커', '는', '얼마', '죠', '#@', '소속', '#', '이요', '이름', '이', '정겹네요', '#@', '소속', '#', '공원', '근처', '에', '맛있는', '분식집', '추천', '해', '주세요', '실내', '에도', '다양하게', '준비', '되어', '있네요', '이번', '에', '김포', '여행', '을', '가려고', '하는데요', '아무', '것', '도', '없는', '줄', '알았어요', '#@', '소속', '#', '맞습니다', '배', '를', '타는', '시간', '이', '있나요', '절매', '나루터', '에서', '진', '탄', '나루터', '까지', '하는', '코스', '맞지요', '#@', '소속', '#', '에서', '받으면', '되는', '거', '죠', '그건', '따로', '못', '보긴', '했는데', '수요일', '만', '쉬는', '건가', '요', '영월', '10', '경', '중', '에', '사람', '이', '지은', '건축물', '같은', '거', '말구요', '강가', '따라서', '풍경', '볼', '수', '있는', '곳', '추천', '좀', '해주세요', '#@', '소속', '#', '예약', '할', '수', '있나', '확인', '하려구요', '녹두', '한', '방', '백숙', '은', '가격', '이', '얼마', '예요', '건물', '뒤', '에', '있어서', '주차장', '이', '안', '보였군요', '앞', '에', '차', '들', '은', '뭐', '예요', '체험', '할', '수', '있는', '프로그램', '이', '있습니까', '친절한', '안내', '감사', '드립니다', '강습', '은', '받지', '않고', '장비', '렌탈', '만', '이용', '할', '수', '는', '없나요', '땀', '이', '많은', '편이', '라', '복장', '때문',

In [14]:
# 길이가 0인 단어 삭제
words = [word for word in words if len(word) > 0]

In [15]:
# 중복된 단어 삭제
words = list(set(words))

In [16]:
# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, STA, END, OOV]

In [17]:
len(words)

3654

In [18]:
words

['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 '덜',
 '아기',
 '12만원',
 '보려구요',
 '들어가게',
 '샴푸',
 '있죠',
 '필요',
 '차면',
 '드셔',
 '쉬',
 '마찬가지',
 '얘기',
 '부산',
 '삼국유사',
 '살리는',
 '형',
 '앞',
 '되네요',
 'X',
 '정신',
 '학교',
 '알려주시겠어요',
 'SNS',
 '외국인',
 '있을까',
 '건가',
 '민콜',
 '이십분',
 '이상은',
 '3500원',
 '일원',
 '맞나요',
 '직원',
 '조강',
 '생긴',
 '가능하군요',
 '산',
 '무거운',
 '여행',
 '가지러',
 '넓네요',
 '주시더라고요',
 '봄',
 '싶으시군요',
 '해드려요',
 '모집',
 '입장권',
 '등재',
 '맛있는',
 'OX',
 '멀리',
 '슬라이드',
 '도솔로',
 '예약',
 '치유',
 '바닷가',
 '다음주',
 '신',
 '유모차',
 '부족한가요',
 '수유',
 '인데요',
 '아프진',
 'JEAN',
 '맑게하는',
 '산책',
 '과수원',
 '둘러보시면',
 '하시면서',
 '9천원',
 '정선',
 '좋아하지',
 '차량',
 '정해져',
 '팔천',
 '없지',
 '상태',
 '괜찮나요',
 '그래서',
 '하시는거',
 '빵',
 '슈페리어룸',
 '유라',
 '야할거',
 '우천',
 '않아서요',
 '지질',
 '원하시는건',
 '내려서',
 '보성',
 '하절',
 '놀러',
 '운봉',
 '봄철',
 '그릴',
 '사람',
 '보실',
 '고도',
 '기념품',
 '25만원',
 '창원',
 '봤습니다',
 '갓김치',
 '까페',
 '렌터카',
 '에선',
 '해야겠네요',
 '족욕',
 '오로라',
 '데',
 '브런치',
 '인',
 '기타',
 '야해요',
 '나오더라구요',
 '렌탈',
 '사면',
 '수성구',
 '결정',
 '찾아가서',
 '낚시터',
 '리무진',

In [19]:
# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

In [20]:
# 단어 => 인덱스
word_to_index

{'<PADDING>': 0,
 '<START>': 1,
 '<END>': 2,
 '<OOV>': 3,
 '덜': 4,
 '아기': 5,
 '12만원': 6,
 '보려구요': 7,
 '들어가게': 8,
 '샴푸': 9,
 '있죠': 10,
 '필요': 11,
 '차면': 12,
 '드셔': 13,
 '쉬': 14,
 '마찬가지': 15,
 '얘기': 16,
 '부산': 17,
 '삼국유사': 18,
 '살리는': 19,
 '형': 20,
 '앞': 21,
 '되네요': 22,
 'X': 23,
 '정신': 24,
 '학교': 25,
 '알려주시겠어요': 26,
 'SNS': 27,
 '외국인': 28,
 '있을까': 29,
 '건가': 30,
 '민콜': 31,
 '이십분': 32,
 '이상은': 33,
 '3500원': 34,
 '일원': 35,
 '맞나요': 36,
 '직원': 37,
 '조강': 38,
 '생긴': 39,
 '가능하군요': 40,
 '산': 41,
 '무거운': 42,
 '여행': 43,
 '가지러': 44,
 '넓네요': 45,
 '주시더라고요': 46,
 '봄': 47,
 '싶으시군요': 48,
 '해드려요': 49,
 '모집': 50,
 '입장권': 51,
 '등재': 52,
 '맛있는': 53,
 'OX': 54,
 '멀리': 55,
 '슬라이드': 56,
 '도솔로': 57,
 '예약': 58,
 '치유': 59,
 '바닷가': 60,
 '다음주': 61,
 '신': 62,
 '유모차': 63,
 '부족한가요': 64,
 '수유': 65,
 '인데요': 66,
 '아프진': 67,
 'JEAN': 68,
 '맑게하는': 69,
 '산책': 70,
 '과수원': 71,
 '둘러보시면': 72,
 '하시면서': 73,
 '9천원': 74,
 '정선': 75,
 '좋아하지': 76,
 '차량': 77,
 '정해져': 78,
 '팔천': 79,
 '없지': 80,
 '상태': 81,
 '괜찮나요': 82,
 '그래서': 83,
 '하시는

In [21]:
# 인덱스 => 단어
index_to_word

{0: '<PADDING>',
 1: '<START>',
 2: '<END>',
 3: '<OOV>',
 4: '덜',
 5: '아기',
 6: '12만원',
 7: '보려구요',
 8: '들어가게',
 9: '샴푸',
 10: '있죠',
 11: '필요',
 12: '차면',
 13: '드셔',
 14: '쉬',
 15: '마찬가지',
 16: '얘기',
 17: '부산',
 18: '삼국유사',
 19: '살리는',
 20: '형',
 21: '앞',
 22: '되네요',
 23: 'X',
 24: '정신',
 25: '학교',
 26: '알려주시겠어요',
 27: 'SNS',
 28: '외국인',
 29: '있을까',
 30: '건가',
 31: '민콜',
 32: '이십분',
 33: '이상은',
 34: '3500원',
 35: '일원',
 36: '맞나요',
 37: '직원',
 38: '조강',
 39: '생긴',
 40: '가능하군요',
 41: '산',
 42: '무거운',
 43: '여행',
 44: '가지러',
 45: '넓네요',
 46: '주시더라고요',
 47: '봄',
 48: '싶으시군요',
 49: '해드려요',
 50: '모집',
 51: '입장권',
 52: '등재',
 53: '맛있는',
 54: 'OX',
 55: '멀리',
 56: '슬라이드',
 57: '도솔로',
 58: '예약',
 59: '치유',
 60: '바닷가',
 61: '다음주',
 62: '신',
 63: '유모차',
 64: '부족한가요',
 65: '수유',
 66: '인데요',
 67: '아프진',
 68: 'JEAN',
 69: '맑게하는',
 70: '산책',
 71: '과수원',
 72: '둘러보시면',
 73: '하시면서',
 74: '9천원',
 75: '정선',
 76: '좋아하지',
 77: '차량',
 78: '정해져',
 79: '팔천',
 80: '없지',
 81: '상태',
 82: '괜찮나요',
 83: '그래서',
 84: 

In [22]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type): 
    
    sentences_index = []
    
    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []
        
        # 디코더 입력일 경우 맨 앞에 START 태그 추가
        if type == DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])
        
        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                # 사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END 태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
            else:
                sentence_index += [vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]
            
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]
        
        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)

    return np.asarray(sentences_index)

In [23]:
# 인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)
x_encoder[0]

array([ 247, 1577, 3119, 1956, 2855, 2043, 1657, 3016,  212, 1089,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [24]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)
x_decoder[0]

array([   1, 2846, 3565, 1308, 2855, 2404, 3565, 1575, 2855, 3443, 3565,
        698, 2855, 2352,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [25]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer, word_to_index, DECODER_TARGET)
y_decoder[0]

array([2846, 3565, 1308, 2855, 2404, 3565, 1575, 2855, 3443, 3565,  698,
       2855, 2352,    2,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [26]:
# 원핫인코딩 초기화
one_hot_data = np.zeros((len(y_decoder), max_sequences, len(words)), dtype=np.int8)

# 디코더 목표를 원핫인코딩으로 변환
# 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
for i, sequence in enumerate(y_decoder):
    for j, index in enumerate(sequence):
        one_hot_data[i, j, index] = 1

# 디코더 목표 설정
y_decoder = one_hot_data

# 첫 번째 디코더 목표 출력
y_decoder[0]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [27]:
# 임베딩 벡터 차원
embedding_dim = 100

# LSTM 히든레이어 차원
lstm_hidden_dim = 128

encoder_inputs = layers.Input(shape=(None,))

encoder_outputs = layers.Embedding(len(words), embedding_dim)(encoder_inputs)

# return_state가 True면 상태값 리턴
# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
encoder_outputs, state_h, state_c = layers.LSTM(lstm_hidden_dim,
                                                dropout=0.1,
                                                recurrent_dropout=0.5,
                                                return_state=True)(encoder_outputs)
# recurrent_dropout : 순환 드롭아웃 - 과거가 아닌 현재 Input에 영향을 받는 Parameter에만 Dropout 적용

# 히든 상태와 셀 상태를 하나로 묶음
encoder_states = [state_h, state_c]

In [28]:
# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
decoder_embedding = layers.Embedding(len(words), embedding_dim)
decoder_outputs = decoder_embedding(decoder_inputs)

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_lstm = layers.LSTM(lstm_hidden_dim,
                           dropout=0.1,
                           recurrent_dropout=0.5,
                           return_state=True,
                           return_sequences=True)

# initial_state를 인코더의 상태로 초기화
decoder_outputs, _, _ = decoder_lstm(decoder_outputs,
                                     initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = layers.Dense(len(words), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [29]:
#--------------------------------------------
# 훈련 모델 정의
#--------------------------------------------

# 입력과 출력으로 함수형 API 모델 생성
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [30]:
# 훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
encoder_model = models.Model(encoder_inputs, encoder_states)

In [31]:
# 예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
# 매번 이전 디코더 상태를 입력으로 받아서 새로 설정
decoder_state_input_h = layers.Input(shape=(lstm_hidden_dim,))
decoder_state_input_c = layers.Input(shape=(lstm_hidden_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]    

# 임베딩 레이어
decoder_outputs = decoder_embedding(decoder_inputs)

# LSTM 레이어
decoder_outputs, state_h, state_c = decoder_lstm(decoder_outputs,
                                                 initial_state=decoder_states_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_h, state_c]

# Dense 레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = models.Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

In [32]:
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break
        elif vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence += vocabulary[OOV_INDEX]
            
        # 빈칸 추가
        sentence += ' '

    return sentence

In [33]:
# 훈련 시작
history = model.fit([x_encoder, x_decoder],
                    y_decoder,
                    epochs=1500,
                    batch_size=64)

Epoch 1/1500
16/16 [==============================] - 7s 175ms/step - loss: 5.3275 - acc: 0.5961
Epoch 2/1500
16/16 [==============================] - 2s 153ms/step - loss: 2.8502 - acc: 0.6377
Epoch 3/1500
16/16 [==============================] - 3s 159ms/step - loss: 2.6365 - acc: 0.6377
Epoch 4/1500
16/16 [==============================] - 3s 157ms/step - loss: 2.5293 - acc: 0.6377
Epoch 5/1500
16/16 [==============================] - 3s 160ms/step - loss: 2.4008 - acc: 0.6418
Epoch 6/1500
16/16 [==============================] - 3s 157ms/step - loss: 2.3266 - acc: 0.6506
Epoch 7/1500
16/16 [==============================] - 3s 162ms/step - loss: 2.2755 - acc: 0.6590
Epoch 8/1500
16/16 [==============================] - 3s 163ms/step - loss: 2.2400 - acc: 0.6656
Epoch 9/1500
16/16 [==============================] - 3s 163ms/step - loss: 2.2084 - acc: 0.6743
Epoch 10/1500
16/16 [==============================] - 3s 159ms/step - loss: 2.1796 - acc: 0.6777
Epoch 11/1500
16/16 [========

In [34]:
# 문장 예측 테스트
input_encoder = x_encoder[2].reshape(1, x_encoder[2].shape[0])
input_decoder = x_decoder[2].reshape(1, x_decoder[2].shape[0])
results = model.predict([input_encoder, input_decoder])

# 결과의 원핫인코딩 형식을 인덱스로 변환
indexs = np.argmax(results[0], 1) 

# 인덱스를 문장으로 변환
sentence = convert_index_to_text(indexs, index_to_word)
print(sentence)

1/1 [==============================] - 1s 618ms/step
국물 떡볶이 가 있는 #@ 소속 # 을 추천 해 드립니다 


In [35]:
# 모델 저장
encoder_model.save('./model/seq2seq_chatbot_encoder_model.h5')
decoder_model.save('./model/seq2seq_chatbot_decoder_model.h5')

# 인덱스 저장
with open('./model/word_to_index.pkl', 'wb') as f:
    pickle.dump(word_to_index, f, pickle.HIGHEST_PROTOCOL)
with open('./model/index_to_word.pkl', 'wb') as f:
    pickle.dump(index_to_word, f, pickle.HIGHEST_PROTOCOL) 

In [36]:
# 모델 파일 로드
encoder_model = models.load_model('./model/seq2seq_chatbot_encoder_model.h5')
decoder_model = models.load_model('./model/seq2seq_chatbot_decoder_model.h5')

# 인덱스 파일 로드
with open('./model/word_to_index.pkl', 'rb') as f:
    word_to_index = pickle.load(f)
with open('./model/index_to_word.pkl', 'rb') as f:
    index_to_word = pickle.load(f)    

In [37]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    
    return input_seq

In [38]:
# 텍스트 생성
def generate_text(input_seq):
    
    # 입력을 인코더에 넣어 마지막 상태 구함
    states = encoder_model.predict(input_seq)

    # 목표 시퀀스 초기화
    target_seq = np.zeros((1, 1))
    
    # 목표 시퀀스의 첫 번째에 <START> 태그 추가
    target_seq[0, 0] = STA_INDEX
    
    # 인덱스 초기화
    indexs = []
    
    # 디코더 타임 스텝 반복
    while 1:
        # 디코더로 현재 타임 스텝 출력 구함
        # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
        decoder_outputs, state_h, state_c = decoder_model.predict(
                                                [target_seq] + states)

        # 결과의 원핫인코딩 형식을 인덱스로 변환
        index = np.argmax(decoder_outputs[0, 0, :])
        indexs.append(index)
        
        # 종료 검사
        if index == END_INDEX or len(indexs) >= max_sequences:
            break

        # 목표 시퀀스를 바로 이전의 출력으로 설정
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = index
        
        # 디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [state_h, state_c]

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
        
    return sentence

In [39]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('문어도 구워먹을 수 있나요')

# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence

1/1 [==============================] - 0s 14ms/step


'저희 는 문어 와 함께 닭갈비 도 맥 반석 에 구워 드실 수 있습니다 '

In [40]:
input_seq = make_predict_input('승마체험 정보 알려주세요')

sentence = generate_text(input_seq)
sentence

1/1 [==============================] - 0s 16ms/step


'지금 계시는 위치 가 어디 쯤 되실까 요 '

In [41]:
input_seq = make_predict_input('언제 영업하나요')

sentence = generate_text(input_seq)
sentence

1/1 [==============================] - 0s 16ms/step


'신청 은 저희 가 받는건 아니고 학교 에서 선착순 으로 신청 가능하세요 '